In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
from scipy.special import expi

%matplotlib inline

/home/junpenglao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
Cs = 2.5E6
rb = 0.0825
Leff = 1.9
Rb = 0.162
q = 50.
t = np.array([i*3600 for i in range(51)], dtype=np.float64)
temp_mea = -(q/(4.0*Leff))*expi(-Cs*(rb**2)/(4.0*t*Leff)) + 6

In [9]:
temp_mea

array([ 6.        ,  8.86120712, 11.78449735, 13.85121381, 15.43202114,
       16.70915916, 17.77974513, 18.70101745, 19.50940995, 20.22950716,
       20.8786668 , 21.46959403, 22.01186545, 22.51287708, 22.97845912,
       23.41328841, 23.82117363, 24.20525757, 24.56816367, 24.91210446,
       25.2389631 , 25.5503556 , 25.84767909, 26.13214961, 26.4048321 ,
       26.66666446, 26.91847694, 27.16100803, 27.3949175 , 27.62079718,
       27.83918001, 28.05054757, 28.2553365 , 28.45394389, 28.64673195,
       28.83403196, 29.0161477 , 29.19335839, 29.36592132, 29.53407403,
       29.69803631, 29.85801192, 30.01419011, 30.16674698, 30.31584665,
       30.46164232, 30.60427724, 30.74388555, 30.88059301, 31.01451771,
       31.14577069])

In [10]:
import theano
import theano.tensor as tt

from theano.compile.ops import as_op

@as_op(itypes=[tt.dscalar, tt.dvector, tt.dscalar, tt.dscalar, tt.dscalar],
       otypes=[tt.dvector])
def ils(q, t, Cs, rb, Leff):
    temp_bore = -(q/(4.0*Leff))*expi(-Cs*(rb**2)/(4.0*t*Leff))
    return temp_bore

In [11]:
basic_model = pm.Model()

with basic_model:

    Leff = pm.Normal('Leff', mu=1.8, sd=0.5)
    Rb = pm.Normal('Rb', mu=0.16, sd=0.1) 

    temp_cal = ils(q, t, Cs, rb, Leff) + Rb*q

    Y_obs = pm.Normal('Y_obs', mu=temp_cal, sd=0.4, observed=temp_mea)

AttributeError: 'float' object has no attribute 'type'

In [12]:
basic_model = pm.Model()
q_shared, t_shared, Cs_shared, rb_shared = theano.shared(q), theano.shared(t), theano.shared(Cs), theano.shared(rb)
with basic_model:

    Leff = pm.Normal('Leff', mu=1.8, sd=0.5)
    Rb = pm.Normal('Rb', mu=0.16, sd=0.1) 

    temp_cal = ils(q_shared, t_shared, Cs_shared, rb_shared, Leff) + Rb*q

    Y_obs = pm.Normal('Y_obs', mu=temp_cal, sd=0.4, observed=temp_mea)